In [4]:
#Import Modules 
import os 
import csv 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sts
import numpy as np
import gmaps
import requests
from citipy import citipy

In [5]:
#Open Wine File 
csv_path = 'Resources/winemag-data.csv'

In [6]:
#Read wine data 
wine_data_pd = pd.read_csv(csv_path)

#Cleaning Data 
clean_data_pd = wine_data_pd.drop(['taster_name', 'taster_twitter_handle'], axis=1)
clean_data_pd = clean_data_pd.dropna(subset=['price', 'region_1','designation','points','price','title','description','variety'])
clean_data_pd

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
...,...,...,...,...,...,...,...,...,...,...,...,...
129959,129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,NaN,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim
129962,129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,NaN,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano
129965,129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,NaN,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser


In [7]:
us_data = clean_data_pd.dropna()
us_data = us_data[us_data['country']=='US']
us_data

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
33,33,US,"Rustic and dry, this has flavors of berries, c...",Puma Springs Vineyard,86,50.0,California,Dry Creek Valley,Sonoma,Envolve 2010 Puma Springs Vineyard Red (Dry Cr...,Red Blend,Envolve
...,...,...,...,...,...,...,...,...,...,...,...,...
129920,129920,US,"Shows the clean, citrus acid juiciness and sil...",Fiddlestix Vineyard,91,48.0,California,Sta. Rita Hills,Central Coast,Pali 2006 Fiddlestix Vineyard Pinot Noir (Sta....,Pinot Noir,Pali
129926,129926,US,This pure Syrah from Reininger's estate vineya...,Ash Hollow Vineyard,91,41.0,Washington,Walla Walla Valley (WA),Columbia Valley,Reininger 2005 Ash Hollow Vineyard Syrah (Wall...,Syrah,Reininger
129945,129945,US,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,90,20.0,California,Santa Ynez Valley,Central Coast,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino
129949,129949,US,There's no bones about the use of oak in this ...,Barrel Fermented,90,35.0,California,Napa Valley,Napa,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs


In [8]:
#Finding the average score for each winery 
grouped_wine = us_data.groupby('winery')

#Finding the the province of each winery
province_wine = pd.DataFrame(grouped_wine['region_2'].unique())

country_wine = pd.DataFrame(grouped_wine['country'].unique())

mean_wine = pd.DataFrame(grouped_wine['points'].mean())

#Finding the average of each winery
#mean_wine = pd.DataFrame(grouped_wine)

wine_pd = pd.merge(province_wine, country_wine, on = 'winery')

#Merging the tables together 
wine_pd = pd.merge(wine_pd, mean_wine, on = 'winery')



#Resetting the Index
wine_pd = wine_pd.reset_index()

#Setting Province as a string 
wine_pd['region_2'] = wine_pd['region_2'].str[0]

wine_pd['country'] = wine_pd['country'].str[0]



In [9]:
#Displaying the Data Frame 
wine_pd

,winery,region_2,country,points
0,10 Knots,Central Coast,US,82.000000
1,100 Percent Wine,California Other,US,86.500000
2,1000 Stories,California Other,US,90.000000
3,12C Wines,Napa,US,92.600000
4,14 Hands,Columbia Valley,US,87.820513
...,...,...,...,...
3699,un4seen,California Other,US,85.000000
3700,Écluse,Central Coast,US,91.411765
3701,Élevée Winegrowers,Willamette Valley,US,93.000000
3702,Órale,Central Coast,US,91.000000


In [10]:
#Finding the wrost wineries among the pack 
worst_us_wineries = wine_pd.nsmallest(50, 'points', keep = 'last')

#Displaying the 50 wrost wineries 
worst_us_wineries

,winery,region_2,country,points
2752,Ronan,Central Coast,US,80.000000
2290,Nine x Nine,Central Valley,US,80.000000
2258,Namasté Vineyards,Willamette Valley,US,80.000000
2222,Moss Roxx,Central Valley,US,80.000000
2189,Mission Trail,Central Coast,US,80.000000
1462,Hatcher,Sierra Foothills,US,80.000000
1200,Finnegan's Lake,California Other,US,80.000000
626,Catharine Valley,Finger Lakes,US,80.000000
546,California's Jewel,Central Valley,US,80.000000
394,Bloomfield,Central Coast,US,80.000000


In [11]:
#Finding the best wineries among the pack 
best_us_wineries = wine_pd.nlargest(50, 'points', keep = 'last')

#Displaying the 10 wrost wineries 
best_us_wineries

,winery,region_2,country,points
164,Araujo,Napa,US,98.000000
2259,Napa Angel,Napa,US,97.000000
3655,Yao Ming,Napa,US,96.333333
777,Colgin,Napa,US,96.333333
2892,Semper,Sonoma,US,96.000000
2684,Reynolds Family Winery,Napa,US,96.000000
1927,Lewis,Napa,US,96.000000
888,Dalla Valle,Napa,US,96.000000
876,D'Alfonso-Curran,Central Coast,US,96.000000
643,Celani Family Vineyards,Napa,US,96.000000


In [12]:
region_2 = worst_us_wineries['region_2']
country = worst_us_wineries['country']

location = zip(region_2, country)

lat = []
lng = []
fips = []
city_list = []

api_key = 'd44516f8f6db4dab8c39e341cbdd2719' 

In [13]:
#for location in location: 
city = 'Napa'
country = 'US'
#url = (f'https://api.opencagedata.com/geocode/v1/json?q={location[0]}%2C%20{location[1]}&key={api_key}')
url = (f'https://api.opencagedata.com/geocode/v1/json?q={city}%2C%20{country}&key={api_key}')
response = requests.get(url)
cordinate_data = response.json()

cordinate_data['results'][0]['annotations']["FIPS"]
#lat.append(cordinate_data['results'][0]['geometry']['lat'])
#lng.append(cordinate_data['results'][0]['geometry']['lng'])

{'county': '06055', 'state': '06'}

In [14]:
for location in location: 
    try:
        url = (f'https://api.opencagedata.com/geocode/v1/json?q={location[0]}%2C%20{location[1]}&key={api_key}')
        response = requests.get(url)
        cordinate_data = response.json()
        lat.append(cordinate_data['results'][0]['geometry']['lat'])
        lng.append(cordinate_data['results'][0]['geometry']['lng'])
        fips.append(cordinate_data['results'][0]['annotations']['FIPS']['county'])
    except KeyError:
        print('no county')

no county
no county
no county
no county
no county
no county
no county
no county
no county


In [15]:
print(fips)

['06077', '41005', '06077', '04013', '06057', '16037', '06077', '06097', '06057', '06073', '06057', '06057', '06077', '06057', '04013', '36103', '06077', '41005', '36071', '06097', '06057', '06055', '06057', '06097', '06077', '06077', '45079', '06057', '06055', '06057', '06055', '06057', '06055', '06057', '04013', '45079', '16037', '06057', '06057', '06097', '45079']


In [ ]:
list_cities = region_1.to_list()

In [ ]:
weather_cord = {'latitude':lat,
               'longitude':lng}


weather_cord = pd.DataFrame(weather_cord)
lat = weather_cord['latitude']
lng = weather_cord['longitude']

cordinates = zip(lat,lng)

In [ ]:
city_list = []

for cordinates in cordinates: 
    city = citipy.nearest_city(cordinates[0], cordinates[1]).city_name
    city_list.append(city)

#Finding the length of the list of cities 
len(city_list)

In [ ]:
weather_cord['City']= city_list
weather_cord

In [ ]:
rainfall = []

weather_api_key = 'e7909cb56283b125242b31247d3922d8'
#city = weather_cord['City']

city = 'greenfield'

url =  'http://api.openweathermap.org/data/2.5/weather?'
#for city in city:
    
query_url = url + 'appid=' + weather_api_key + '&q=' + city + '&units=imperial'
weather_response = requests.get(query_url)
weather_data = weather_response.json()

weather_data